#### 신용카드 사용자들의 개인 신상정보 데이터로 사용자의 신용카드 대금 연체 정도를 예측

##### 평가산식 : Logloss
Public 평가 : 테스트 데이터 중 랜덤 샘플 된 50%로 채점, 대회 기간 중 공개

Private 평가 : 나머지 50 % 테스트 데이터로 채점, 대회 종료 직후 공개

#### 데이터 변수 설명

In [1]:
# index : 인덱스
# gender: 성별
# car: 차량 소유 여부
# reality: 부동산 소유 여부
# child_num: 자녀 수
# income_total: 연간 소득
# income_type: 소득 분류 
# edu_type: 교육 수준 
# family_type: 결혼 여부 
# house_type: 생활 방식 
# DAYS_BIRTH: 출생일 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미
# DAYS_EMPLOYED: 업무 시작일 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미	(양수 값은 고용되지 않은 상태를 의미함)
# FLAG_MOBIL: 핸드폰 소유 여부
# work_phone: 업무용 전화 소유 여부
# phone: 전화 소유 여부
# email: 이메일 소유 여부
# occyp_type: 직업 유형													
# family_size: 가족 규모
# begin_month: 신용카드 발급 월
# 이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미
# credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도 -> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

#### 예측해야 할 칼럼 : credit

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
display(train,test)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,-5434,1,1,1,0,Accountants,2.0,-19.0
9996,36453,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,-1315,1,1,0,0,Laborers,2.0,-34.0
9997,36454,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,-14018,1,0,0,0,Medicine staff,2.0,-55.0
9998,36455,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-16541,-1085,1,0,1,0,NaN,2.0,-33.0


In [4]:
all_data = pd.concat([train,test])

#### 파생변수

In [5]:
# income(연봉) / DAYS_EMPLOYED = 연봉상승률
all_data["pay"] = all_data["income_total"] / all_data["DAYS_EMPLOYED"]

In [6]:
# income(연봉) / family_size -> 부양될 가족이 너무 많다면 좋지 않다.
all_data["family_pay"] = all_data["income_total"] / all_data["family_size"]

In [7]:
#income *  DAYS_EMPLOYED -> 이 사람이 지금까지 얼마나 벌었는가?
all_data["year_pay"] = all_data['income_total'] * all_data["DAYS_EMPLOYED"]

In [8]:
# 카테고리 타입 끼리 엮어주기
all_data["occyp_type"] = all_data["occyp_type"].fillna("null")
all_data["edu_job"] = all_data["edu_type"] + "_" + all_data["occyp_type"]
all_data["edu_income"] = all_data["edu_type"] + "_" + all_data["income_type"]
all_data["family_house"] = all_data["family_type"] + "_" + all_data["house_type"]

In [9]:
# 교육 수준 기준으로 연간 소득의 통계량 추가
#pd.options.display.max_columns = 999
edu_income = all_data.groupby("edu_type")["income_total"].agg(["count","mean","median","std","min","max","sum"])
all_data = all_data.join(edu_income,on="edu_type")

In [10]:
# 직업 유형을 알 수 없으며 결혼한 경우
all_data["null_marry"] = np.where(((all_data["occyp_type"] == "null") & (all_data["family_type"] == "Married")),1,0)

In [11]:
# 소득분류가 "Working"이면서, 연간소득이 평균 미만인 경우
all_data["worst_income"] = np.where(((all_data["income_type"] == "Working") & (all_data["income_total"] < 186685)),1,0)

In [12]:
# 재산
all_data["has_job"] = all_data["DAYS_EMPLOYED"] != 365243
all_data["money"] = all_data["car"].replace({"Y":1,"N":0}) + all_data["reality"].replace({"Y":1,"N":0}) * 2 + all_data["phone"] + all_data["has_job"] 

In [13]:
all_data = all_data.drop(columns=["has_job"])

In [14]:
# 개인마다 평균을 기준으로 좀 더 번다?! 벌지 못한다?!
all_data["income_diff"] = all_data["income_total"] - all_data["mean"] * 1.5

In [15]:
# 연봉 -> 표준편차
all_data["income_std"] = np.sqrt(((all_data["income_total"] - all_data["income_total"].mean())**2) / len(all_data["income_total"]))

In [16]:
# 나이가 많은 데 ..직장에도 오래 있었다면?
all_data["DAYS_plus"] = all_data["DAYS_BIRTH"] + all_data["DAYS_EMPLOYED"] 

In [17]:
all_data["DAYS_Square"] = np.sqrt(all_data["DAYS_BIRTH"] ** 2 + all_data["DAYS_EMPLOYED"] ** 2)

In [18]:
all_data["DAYS_Square_minus"] = np.sqrt(all_data["DAYS_BIRTH"] ** 2 - all_data["DAYS_EMPLOYED"] ** 2)

/Users/genie/opt/anaconda3/envs/genie/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [19]:
# 취업을 얼마나 빨리 했는지 
all_data["NoEmployed"] = all_data["DAYS_BIRTH"] - all_data["DAYS_EMPLOYED"] * 2

In [20]:
# 카테고리 타입 끼리 엮어 주기
all_data["gender_car_reality"] = all_data["gender"] + "_" + all_data["car"] + "_" + all_data["reality"]

In [21]:
# "email"칼럼을 카테고리 타입으로 변환 동시에 카테고리 타입으로 엮어 주기
all_data["job_email"] = all_data["occyp_type"] + "_" + all_data["email"].astype("str")

In [22]:
all_data2 = all_data.drop(columns=["index","credit","FLAG_MOBIL"])

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,worst_income,money,income_diff,income_std,DAYS_plus,DAYS_Square,DAYS_Square_minus,NoEmployed,gender_car_reality,job_email
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,...,0,1,-133100.499088,82.824439,-18608,14675.042828,13076.984362,-4481,F_N_N,null_0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,...,0,3,-10027.979749,318.504070,-12920,11483.727618,11275.318177,-8300,F_N_Y,Laborers_1
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,...,0,5,114399.500912,1379.062410,-23521,19595.252614,18564.838082,-10219,M_Y_Y,Managers_0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,...,0,4,-55027.979749,82.824439,-17180,15232.340858,14942.264889,-10904,F_N_Y,Sales staff_0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,...,0,4,-178100.499088,152.855192,-17142,15183.622559,14888.933609,-10827,F_Y_Y,Managers_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,...,0,5,-99994.787234,82.824439,-24027,19370.802900,17781.206174,-7725,F_Y_Y,Accountants_0
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,...,0,4,-55027.979749,82.824439,-12201,10965.136616,10806.283866,-8256,M_Y_Y,Laborers_0
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,...,0,3,34972.020251,554.183701,-35034,25262.157073,15657.839315,7020,F_N_Y,Medicine staff_0
9998,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-16541,...,0,3,-77527.979749,35.015377,-17626,16576.546866,16505.376579,-14371,F_Y_N,null_0


In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data2.columns[all_data2.dtypes == "object"]:
    all_data2[i] = le.fit_transform(all_data2[i])

In [24]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

#### 베이지안 튜닝

In [25]:
# import xgboost as xgb
# params = {"learning_rate" : 0.02, "num_class" : 3, "objective" : "multi:softprob", "eval_metric" : "mlogloss","reg_alpha" : 1e-05}
# xgb_train = xgb.DMatrix(train2,train["credit"]) 
# result_xgb = xgb.cv(params,xgb_train, num_boost_round = 50, early_stopping_rounds = 50, nfold = 5, stratified = True)

In [26]:
# def xgb_bayesian(max_depth,colsample_bytree,subsample,colsample_bylevel,learning_rate):
#     params = {"learning_rate" : learning_rate, "num_class" : 3, 
#               "objective" : "multi:softprob", "eval_metric" : "mlogloss",
#               "max_depth": int(max_depth),"colsample_bytree" : colsample_bytree,
#               "subsample" : subsample, "colsample_bylevel" : colsample_bylevel,
#               }   
#     xgb_train = xgb.DMatrix(train2,train["credit"]) 
#     result_xgb = xgb.cv(params,xgb_train, num_boost_round = 50000, 
#                         early_stopping_rounds = 50, nfold = 4, stratified = True)
#     return -result_xgb["test-mlogloss-mean"].min()

In [27]:
# from bayes_opt import BayesianOptimization
# bo = BayesianOptimization(xgb_bayesian, {"max_depth":(10,20),"colsample_bytree":(0.2,0.7),"subsample":(0.4,0.8),
#                                          "colsample_bylevel":(0.3,1),"learning_rate":(0.02,0.05)})
# bo.maximize(init_points = 20, n_iter = 500)

#### 베이지안 튜닝을 통해 파라미터를 찾아 모델 파라미터에 넣어주었다.

In [28]:
n = 0
result = 0 
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 20,random_state = 82, shuffle = True) 
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators= 50000,learning_rate = 0.02,max_depth = 19,subsample=0.7,colsample_bytree=0.8,
                    colsample_bylevel=0.6,colsample_bynode = 0.3,reg_alpha=1e-05,reg_lambda = 1.5)
best_score = 0
for train_index, valid_index in skf.split(train2,train["credit"]):
    X_train , X_valid = train2.iloc[train_index] , train2.iloc[valid_index]
    y_train , y_valid = train["credit"].iloc[train_index] , train["credit"].iloc[valid_index]    
    xgb.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds = 50,verbose = 0) 
    n += 1
    print(f"{n}회")
    best_score += xgb.best_score/20
    result += xgb.predict_proba(test2)/20

/Users/genie/opt/anaconda3/envs/genie/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:04:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1회
[10:05:59] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2회
[10:07:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
3회
[10:08:26] WARNING: /Users/runner/miniforge3/conda-bld/

In [29]:
result

array([[0.07190429, 0.15396403, 0.7741317 ],
       [0.30692482, 0.23515235, 0.45792285],
       [0.03184037, 0.03784555, 0.93031406],
       ...,
       [0.03467548, 0.08883516, 0.8764893 ],
       [0.16658047, 0.37093306, 0.4624865 ],
       [0.08587933, 0.23723342, 0.6768872 ]], dtype=float32)

#### 내부 평가셋 점수

In [30]:
best_score 

0.67976625

In [31]:
sub = pd.read_csv("sample_submission.csv")
sub

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [32]:
sub.iloc[:,1:] = result
sub

,index,0,1,2
0,26457,0.071904,0.153964,0.774132
1,26458,0.306925,0.235152,0.457923
2,26459,0.031840,0.037846,0.930314
3,26460,0.076388,0.073799,0.849813
4,26461,0.068543,0.183178,0.748279
...,...,...,...,...
9995,36452,0.099606,0.256957,0.643437
9996,36453,0.307418,0.266951,0.425632
9997,36454,0.034675,0.088835,0.876489
9998,36455,0.166580,0.370933,0.462487


In [33]:
sub.to_csv("final.csv",index=False)

#### [최종결과] 참가자수 : 1790명, 제출자수 : 714명, 등수 : 48등, private점수 : 0.66726 상위 7%